## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0.3,-0.4,0), (0,0,np.pi/2)),
                INDY_IP),
    RobotConfig(1, RobotType.panda, ((-0.3,-0.4,0), (0,0,np.pi/2)),
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

connection_list
[False, False]


In [3]:
from pkg.detector.aruco.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
stereo.calibrate()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming


[(array([[1.82983423e+03, 0.00000000e+00, 1.91572046e+03],
         [0.00000000e+00, 1.82983423e+03, 1.09876086e+03],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([ 7.09966481e-01, -2.73409390e+00,  1.45804870e-03, -3.24774766e-04,
          1.44911301e+00,  5.84310412e-01, -2.56374550e+00,  1.38472950e+00])),
 (array([[1.39560388e+03, 0.00000000e+00, 9.62751587e+02],
         [0.00000000e+00, 1.39531934e+03, 5.47687012e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([0., 0., 0., 0., 0.])),
 array([[ 0.82852983, -0.02117649, -0.5595443 ,  0.64588934],
        [ 0.16892579,  0.96217924,  0.21371762, -0.16182286],
        [ 0.5338561 , -0.2715929 ,  0.8007715 ,  0.09055868],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],
       dtype=float32)]

## create scene builder

In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo)
s_builder.reset_reference_coord(ref_name="floor")

## detect robot and make geometry scene

In [5]:
xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## init planning pipeline

In [6]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [7]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
Please create a subscriber to the marker
generate table - Geometry
generate table - Object
generate table - Marker
generate table - MarkerGroup
button clicked
button action done
button clicked
button action done
generate table - Handle
generate table - Binder
generate table - Geometry
generate table - Object
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 1950, in full_dispatch_request
 

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/rnb/.local/lib/python2.7/site-packages/dash/dash.py", line 1076, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/home/rnb/.local/lib/python2.7/site-packag

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/rnb/.local/lib/python2.7/site-packages/dash/dash.py", line 1076, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/home/rnb/.local/lib/python2.7/site-packag

generate table - MotionPlan
generate table - TaskPlan
generate table - PlanList
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/rnb/.local/lib/python2.7/site-packages/dash/dash.py", line 1076, in dispatch
    response.set_data(func(*args

In [8]:
# add cam poles
ptems = s_builder.add_poles({"cam0": s_builder.ref_coord_inv[:3,3], "cam1":np.matmul(s_builder.ref_coord_inv, stereo.T_c12)[:3,3]},
                            color=(0.6,0.6,0.6,0.0))

In [9]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

In [10]:
 gscene.NAME_DICT["floor"].dims = (2,1,0.01)

In [11]:
gscene.create_safe(GEOTYPE.BOX, "ceiling", "base_link", (2,1,0.01), 
                   (0.0,0.0,1.1),rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,0.1), display=True, fixed=True, collision=True)
gscene.create_safe(GEOTYPE.BOX, "leftwall", "base_link", (1,1,0.01), 
                   (1.0,0.0,0.5),rpy=(0,np.pi/2,0), 
                   color=(0.8,0.8,0.8,0.1), display=True, fixed=True, collision=True)
gscene.create_safe(GEOTYPE.BOX, "rightwall", "base_link", (1,1,0.01), 
                   (-1.0,0.0,0.5),rpy=(0,np.pi/2,0), 
                   color=(0.8,0.8,0.8,0.1), display=True, fixed=True, collision=True)
gscene.create_safe(GEOTYPE.BOX, "frontwall", "base_link", (2,1,0.01), 
                   (0.0,-0.5,0.5),rpy=(np.pi/2,0,0), 
                   color=(0.8,0.8,0.8,0.1), display=True, fixed=True, collision=True)

## add sweep face

In [12]:
track = s_builder.detect_and_register(item_names=["track"], level_mask=[DetectionLevel.ENVIRONMENT])["track"]
track_face = gscene.copy_from(track, new_name="track_face", collision=False)
track_face.dims = (track.dims[0]-0.22, track.dims[1]-0.12, track.dims[2])
# track.dims = (track.dims[0]-0.32, track.dims[1]-0.12, track.dims[2])

In [13]:
gscene.update_markers_all()

## add wp

In [14]:
TOOL_DIM = [0.06, 0.10]
TRACK_DIM = track_face.dims
TRACK_NUM = np.ceil(np.divide(TRACK_DIM[1]-TOOL_DIM[1], TOOL_DIM[1])).astype(np.int)+1
TRACK_STEP = (TRACK_DIM[1]-TOOL_DIM[1])/(TRACK_NUM-1)
WP_REF_A = -np.subtract(TRACK_DIM[:2], TOOL_DIM[:2])/2
WP_REF_B = np.array([-WP_REF_A[0], WP_REF_A[1]])
TRC_THIC = TRACK_DIM[2]
track_list = []
for i_trc in range(TRACK_NUM):
    wp1 = gscene.create_safe(GEOTYPE.BOX, "wp{}a".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_A+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    wp2 = gscene.create_safe(GEOTYPE.BOX, "wp{}b".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_B+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    line = gscene.create_safe(GEOTYPE.BOX, "trc{}".format(i_trc+1), "base_link", (TRACK_DIM[0]*2, TRACK_DIM[0]*2, 1e-6), 
                               (0,WP_REF_A[1]+TRACK_STEP*i_trc,0), rpy=(np.pi/2,0,0), 
                               color=(0.8,0.2,0.2,0.2), display=False, fixed=True, collision=False, parent="track_face")
    line_vis = gscene.create_safe(GEOTYPE.BOX, "trc{}vis".format(i_trc+1), "base_link", (TRACK_DIM[0], 0.01, 1e-6), 
                               (0,WP_REF_A[1]+TRACK_STEP*i_trc,0), rpy=(np.pi/2,0,0), 
                               color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    track_list.append((wp1, wp2, line))

In [15]:
gscene.update_markers_all()

## add indy tool

In [16]:
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="indy0_fts", link_name="indy0_tcp",
                            center=(0,0,0.02), dims=(0.07,0.07, 0.04), rpy=(0,0,0), color=(0.8,0.8,0.8,1),
                            collision=False)
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="indy0_fts_col", link_name="indy0_tcp",
                            center=(0,0,0.02), dims=(0.11,0.11, 0.04), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5),
                            collision=True)
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="indy0_pole", link_name="indy0_tcp",
                            center=(0,0,0.055), dims=(0.03,0.03, 0.030), rpy=(0,0,0), color=(0.8,0.8,0.8,1),
                            collision=False)
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="indy0_pole_col", link_name="indy0_tcp",
                            center=(0,0,0.055), dims=(0.07,0.07, 0.030), rpy=(0,0,0), color=(0.0,0.8,0.0,0.2),
                            collision=True)

gscene.create_safe(gtype=GEOTYPE.BOX, name="indy0_brushbase", link_name="indy0_tcp",
                            center=(0,0,0.0775), dims=(0.06, 0.14, 0.015), rpy=(0,0,np.pi/4), color=(0.8,0.8,0.8,1),
                            collision=False)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="indy0_tcp",
                            center=(0,0,0.095), dims=(0.05,0.13, 0.015), rpy=(0,0,np.pi/4), color=(1.0,1.0,0.94,1),
                            collision=False)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face_col", link_name="indy0_tcp",
                            center=(0,0,0.080), dims=(0.08,0.18, 0.02), rpy=(0,0,np.pi/4), color=(0.0,0.8,0.0,0.5),
                            collision=True)

## add box

In [17]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

In [18]:
# give clearance from the track surface
CLEARANCE = 1e-3
Ttrack = track.get_tf(crob.home_dict)
Ttrack_inv = SE3_inv(Ttrack)
for gtem in gtem_dict.values():
    verts, _ = gtem.get_vertice_radius_from(crob.home_dict, from_link="base_link")
    verts_loc = np.matmul(Ttrack_inv[:3,:3], verts.transpose())+Ttrack_inv[:3,3:]
    off = track.dims[2]/2-np.min(verts_loc[2,:])+CLEARANCE
    center_loc = np.matmul(Ttrack_inv[:3,:3], gtem.center) + Ttrack_inv[:3,3]
    center_loc_new = center_loc + [0,0,off]
    center_new = np.matmul(Ttrack[:3,:3], center_loc_new) + Ttrack[:3,3]
    gtem.set_offset_tf(center=center_new)

In [19]:
gscene.update_markers_all()

## Register binders

In [20]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool, SweepFramer, FixtureSlot

In [21]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

pscene.create_binder(bname="grip1", gname="grip1", rname="panda1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="track_face", gname="track_face", _type=PlacePlane)
pscene.create_binder(bname="brush_face", gname="brush_face", rname="indy0", _type=SweepFramer, point=(0,0,0.00), 
                     rpy=(0,np.pi,0))

## add objects

In [22]:
from pkg.planning.constraint.constraint_subject import \
        CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepFrame, SweepTask, BoxObject, FixturePoint, AbstractObject

In [23]:
box1 = pscene.create_subject(oname="box1", gname="box1", _type=BoxObject, hexahedral=True)
box2 = pscene.create_subject(oname="box2", gname="box2", _type=BoxObject, hexahedral=True)
# box3= pscene.create_subject(oname="box3", gname="box3", _type=BoxObject, hexahedral=True)

In [24]:
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [25]:
for i_t, track_tem in enumerate(track_list):
    wp1, wp2, tline = track_tem
    sweep1 = pscene.create_subject(oname="sweep{}".format(i_t+1), gname="track_face", _type=SweepLineTask, 
                                 action_points_dict = {wp1.name: SweepFrame(wp1.name, wp1, [0,0,0.005], [0,0,0]),
                                                       wp2.name: SweepFrame(wp2.name, wp2, [0,0,0.005], [0,0,0])},
                                geometry_vertical = tline)

In [26]:
# pscene.remove_object("sweep3")

### planners

In [27]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion(mplan)
ppline.set_sampler(tplan)

## motion filters

In [28]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker

gcheck = GraspChecker(pscene, 
                      end_link_couple_dict= {
                          "indy0_tcp": ["indy0_tcp", "indy0_link6"],
                          "panda1_hand": ["panda1_hand", "panda1_link6"],
                          "base_link":["base_link"]})
rcheck = ReachChecker(pscene)
checkers_all = [gcheck, rcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

In [29]:
mplan.motion_filters = checkers_all

In [30]:
gscene.show_pose(crob.home_pose)

## Set initial condition

In [31]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.update_state(crob.home_pose)
print(initial_state.node)

('track_face', 'track_face', 0, 0, 0)


In [32]:
pscene.subject_name_list

['box1', 'box2', 'sweep1', 'sweep2', 'sweep3']

# Node Sampler

In [33]:
from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler, PenaltyNodeSampler, GrowingSampler

tplan.new_node_sampler = PenaltyNodeSampler(3, 1)
tplan.parent_node_sampler = UniformNodeSampler(3)
# tplan.parent_snode_sampler = GrowingSampler(3)

generate table - Geometry
generate table - Handle
generate table - Bindergenerate table - Object

generate table - MotionPlan
generate table - TaskPlan
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/rnb/.local/lib/python2.7/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/rnb/.local/lib/python2.7/site-package

# CustomRule

In [37]:
from pkg.planning.task.custom_rules.sweep_entrance_control import SweepEntranceControlRule
tplan.custom_rule = SweepEntranceControlRule(pscene)

## Plan

In [38]:
mplan.reset_log(False)
gtimer.reset()
# goal_nodes = [("track_face","track_face")+(2,0,0)]
goal_nodes = [("track_face","track_face")+tuple([2 for _ in range(TRACK_NUM)])]
# goal_nodes = [("floor","floor","floor", 2, 2, 2)]
gtimer.tic("plan")
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
              timeout_loop=200, multiprocess=True, timeout=10, timeout_constrained=50)
gtimer.toc("plan")
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])

Use 20/20 agents
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
transition motion tried: True
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0) = success
branching: 0->1 (0.18/200.0 s, steps/err: 16(127.3458004 ms)/0.00160030947407)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
t

branching: 3->6 (0.82/200.0 s, steps/err: 20(175.565958023 ms)/9.89875722074e-16)
try: 6 - ('grip1', 'track_face', 0, 0, 0)->('grip1', 'track_face', 1, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
try: 6 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
result: 6 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
transition motion tried: True
try: 6 - ('grip1', 'track_face', 0, 0, 0)->

try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
transition motion tried: True
try: 14 - ('track_face', 'grip1', 1, 0, 0)->('track_face', 'grip1', 2, 0, 0)
try: 13 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
result: 11 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0) = success
branching: 11->14 (1.69/200.0 s, steps/err: 16(243.610143661 ms)/0.00134144067217)
result: 13 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
result: 10 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
try: 13 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
try: 11 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
try: 0 - ('track_face', 'track_face',

try: 16 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
transition motion tried: True
transition motion tried: True
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = success
result: 6 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = success
transition motion tried: True
branching: 0->18 (2.29/200.0 s, steps/err: 23(942.373037338 ms)/0.00108834270179)
branching: 6->19 (2.3/200.0 s, steps/err: 28(257.544994354 ms)/0.00184205468181)
result: 15 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0) = success
try: 17 - ('grip1', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
branching: 15->20 (2.32/200.0 s, steps/err: 26(198.648929596 ms)/0.00113192546268)
try: 17 - ('grip1', 'track_face', 0, 0, 0)->('grip1', 'track_face', 1, 0, 0)
transition motion tried: True
try: 18 - ('grip1', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
result: 10 - ('track_face', 'track_fac

branching: 6->33 (2.99/200.0 s, steps/err: 31(1127.54702568 ms)/0.00163878635385)
joint motion tried: True
result: 15 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = success
result: 31 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0) = fail
transition motion tried: True
branching: 15->34 (3.03/200.0 s, steps/err: 35(924.326181412 ms)/8.43330857823e-16)
try: 33 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
try: 33 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
result: 27 - ('grip1', 'track_face', 0, 0, 0)->('grip1', 'track_face', 1, 0, 0) = success
transition motion tried: True
branching: 27->35 (3.07/200.0 s, steps/err: 15(214.497089386 ms)/0.00115067595909)
result: 30 - ('grip1', 'track_face', 0, 0, 0)->('grip1', 'track_face', 1, 0, 0) = success
try: 34 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0)
branching: 30->36 (3.1/200.0 s, steps/err: 15(14

...

try: 423 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0)
branching: 354->423 (74.01/200.0 s, steps/err: 23(770.081996918 ms)/0.00115685855776)
try: 423 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'grip1', 2, 2, 1)
result: 423 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'grip1', 2, 2, 1) = fail
try: 404 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0)
result: 404 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0) = fail
try: 361 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0)
transition motion tried: False
result: 371 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'grip1', 2, 2, 1) = fail
try: 399 - ('grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
result: 399 - ('grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = fail
try: 365 - ('grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
joint motion tried: True
result: 

result: 427 - ('grip1', 'track_face', 2, 2, 2)->('track_face', 'track_face', 2, 2, 2) = fail
try: 427 - ('grip1', 'track_face', 2, 2, 2)->('track_face', 'track_face', 2, 2, 2)
try: 427 - ('grip1', 'track_face', 2, 2, 2)->('track_face', 'track_face', 2, 2, 2)
result: 427 - ('grip1', 'track_face', 2, 2, 2)->('track_face', 'track_face', 2, 2, 2) = fail
transition motion tried: False
result: 427 - ('grip1', 'track_face', 2, 2, 2)->('track_face', 'track_face', 2, 2, 2) = fail
joint motion tried: True
try: 430 - ('grip1', 'track_face', 2, 2, 2)->('track_face', 'track_face', 2, 2, 2)
try: 427 - ('grip1', 'track_face', 2, 2, 2)->('track_face', 'track_face', 2, 2, 2)
result: 387 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'grip1', 2, 2, 1) = fail
result: 432 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0) = success
try: 436 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'grip1', 2, 2, 1)
result: 427 - ('grip1', 'track_face', 2, 2, 2)->('track_face', 'track_face',

joint motion tried: True
transition motion tried: True
======================= terminated 1: first answer acquired ===============================
result: 440 - ('track_face', 'track_face', 2, 2, 2)->('track_face', 'grip1', 2, 2, 2) = success
branching: 440->447 (77.14/200.0 s, steps/err: 23(426.543951035 ms)/0.00103284008937)
======================= terminated 5: first answer acquired from other agent ===============================
======================= terminated 15: first answer acquired ===============================
joint motion tried: True
transition motion tried: True
result: 441 - ('track_face', 'track_face', 2, 2, 2)->('grip1', 'track_face', 2, 2, 2) = success
branching: 441->449 (77.56/200.0 s, steps/err: 94(754.222154617 ms)/0.00134189406989)
======================= terminated 19: first answer acquired from other agent ===============================
======================= terminated 12: first answer acquired ===============================
transition motion tried: Fals

constrained motion tried: True
result: 377 - ('grip1', 'track_face', 2, 2, 1)->('grip1', 'track_face', 2, 2, 2) = success
branching: 377->453 (90.15/200.0 s, steps/err: 48(25723.3309746 ms)/0.00139410490807)
======================= terminated 2: first answer acquired from other agent ===============================


Process Process-11:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "pkg/planning/pipeline.py", line 177, in __search_loop
    self.add_return_motion(snode_new)
  File "pkg/planning/pipeline.py", line 244, in add_return_motion
    display=False, timeout=timeout)
  File "pkg/planning/pipeline.py", line 200, in test_connection
    self.mplan.plan_transition(from_state, to_state, redundancy_dict=redundancy_dict, **kwargs)
  File "pkg/planning/motion/interface.py", line 105, in plan_transition
    redundancy_values=redundancy_values, **kwargs)
  File "pkg/planning/motion/moveit/moveit_planner.py", line 165, in plan_algorithm
    group_name, tuple(to_Q), tuple(from_Q), timeout=timeout_joint)
  File "pkg/planning/motion/moveit/moveit_py.py", line 123, in plan_joint_motion_py
    JointSta

In [39]:
print(gtimer)

plan: 	77014.0 ms/1 = 77013.688 ms (77013.688/77013.688)



In [37]:
# for i, sidx in enumerate([1, 2, 4, 6]):
#     save_json("traj{}.json".format(i), tplan.snode_dict[sidx].traj)
#     gscene.show_motion(tplan.snode_dict[sidx].traj, period=0.05)
        

In [38]:
# import matplotlib.pyplot as plt
# kin = stereo.camera_list[0]
# rs = stereo.camera_list[1]

In [39]:
# plt.figure(figsize=(15,10))
# plt.imshow(kin.get_image())

In [40]:
# plt.figure(figsize=(15,10))
# plt.imshow(rs.get_image())

## play searched plan

In [40]:
ppline.play_schedule(snode_schedule[:2], period=0.001)
pscene.set_object_state(initial_state)
gscene.show_pose(crob.home_pose)
time.sleep(0.5)
gscene.show_pose(crob.home_pose)

In [41]:
ppline.play_schedule(snode_schedule, period=0.03)

In [42]:
# print("schedule length: {}".format(len(snode_schedule)))
# for i_s,  snode in enumerate(snode_schedule):
#     print("{}: {}".format(i_s, snode.state.node))

In [43]:
# save_json("traj.json", snode_schedule[1].traj)

## execute plan

In [44]:
crob.reset_connection([True,True])

connection_list
[True, True]
Connect: Server IP (192.168.0.63)


In [45]:
indy = crob.robot_dict["indy0"]
panda = crob.robot_dict["panda1"]
with indy:
    indy.set_joint_vel_level(1)
    indy.set_task_vel_level(1)

Connect: Server IP (192.168.0.63)


In [46]:
crob.joint_move_make_sure(crob.home_pose)

Connect: Server IP (192.168.0.63)


In [47]:
indy.stop_tracking()

{'stop': True}

In [48]:
crob.grasp({"indy0":False, "panda1": False})

grasp_dict
{'indy0': False, 'panda1': False}
grasp_seq
[('indy0', False), ('panda1', False)]
Connect: Server IP (192.168.0.63)


In [49]:
ppline.execute_schedule(snode_schedule, vel_scale=1, acc_scale=0.2)

binder: track_face
rname: None
binder: track_face
rname: None
binder: None
binder: None
binder: None
grasp_dict
{'indy0': False, 'panda1': False}
grasp_seq
[('indy0', False), ('panda1', False)]
Connect: Server IP (192.168.0.63)
binder: track_face
rname: None
binder: track_face
rname: None
binder: None
binder: None
binder: None
grasp_dict
{'indy0': False, 'panda1': False}
grasp_seq
[('indy0', False), ('panda1', False)]
Connect: Server IP (192.168.0.63)
go
binder: grip1
rname: panda1
binder: track_face
rname: None
binder: None
binder: None
binder: None
grasp_dict
{'indy0': False, 'panda1': True}
grasp_seq
[('panda1', True), ('indy0', False)]
Connect: Server IP (192.168.0.63)
go
binder: grip1
rname: panda1
binder: track_face
rname: None
binder: None
binder: None
binder: None
grasp_dict
{'indy0': False, 'panda1': True}
grasp_seq
[('panda1', True), ('indy0', False)]
Connect: Server IP (192.168.0.63)
go
binder: track_face
rname: None
binder: track_face
rname: None
binder: None
binder: None
b

KeyboardInterrupt: 

In [ ]:
# for i_s, snode in enumerate(snode_schedule[1:]):
#     save_json("traj%02d.json"%(i_s), snode.traj)

In [ ]:
indy = crob.robot_dict["indy0"]
indy.stop_tracking()